In [4]:
import os
import re
import cv2
import fnmatch
import numpy as np
import pandas as pd
import open3d as o3d
import pyvista as pv
import plotly.express as px
import matplotlib.pyplot as plt
import image_stack_to_pcd as nk_module

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def get_path (path: str):
	file_names = os.listdir (path)
	file_path = []

	for file_name in file_names:
		file_path.append (os.path.abspath (f'{path}/{file_name}'))

	return file_path


path = 'bv_df20'
files = get_path (path = path)
files

['/Users/minkota/Library/CloudStorage/OneDrive-UniversityofGeorgia/IRM/3D rec/bv_df20/bv_84_df20.tif',
 '/Users/minkota/Library/CloudStorage/OneDrive-UniversityofGeorgia/IRM/3D rec/bv_df20/bv_85_df20.tif',
 '/Users/minkota/Library/CloudStorage/OneDrive-UniversityofGeorgia/IRM/3D rec/bv_df20/bv_76_df20.tif',
 '/Users/minkota/Library/CloudStorage/OneDrive-UniversityofGeorgia/IRM/3D rec/bv_df20/bv_77_df20.tif',
 '/Users/minkota/Library/CloudStorage/OneDrive-UniversityofGeorgia/IRM/3D rec/bv_df20/bv_99_df20.tif',
 '/Users/minkota/Library/CloudStorage/OneDrive-UniversityofGeorgia/IRM/3D rec/bv_df20/bv_98_df20.tif',
 '/Users/minkota/Library/CloudStorage/OneDrive-UniversityofGeorgia/IRM/3D rec/bv_df20/bv_94_df20.tif',
 '/Users/minkota/Library/CloudStorage/OneDrive-UniversityofGeorgia/IRM/3D rec/bv_df20/bv_83_df20.tif',
 '/Users/minkota/Library/CloudStorage/OneDrive-UniversityofGeorgia/IRM/3D rec/bv_df20/bv_82_df20.tif',
 '/Users/minkota/Library/CloudStorage/OneDrive-UniversityofGeorgia/IRM/3D

# NK_module

In [7]:
path_win = 'E:\\OneDrive - University of Georgia\\IRM\\3D rec\\bv'
path_mac = '/Users/minkota/Library/CloudStorage/OneDrive-UniversityofGeorgia/IRM/3D rec/bv_df20'

STACK = nk_module.process_files_in_folder (
		folder_path = path_mac
		)

Opened folder:
/Users/minkota/Library/CloudStorage/OneDrive-UniversityofGeorgia/IRM/3D rec/bv_df20
There are 25 files in the folder

IMAGE PROCESSING STAGE

[INFO] File [25/25] is processing
PROCESSING COMPLETED


In [5]:
reconstructed_df = nk_module.tissue_reconstruction (stack = STACK)

Completedex is: [25/25]


In [6]:
pd_reconstructed_df = pd.DataFrame (
		data = reconstructed_df,
		columns = ['x', 'y', 'z']
		)

const_map = 23_082
x_coefficient = pd_reconstructed_df.x.max() / const_map
y_coefficient = pd_reconstructed_df.y.max() / const_map
z_coefficient = 30

pd_reconstructed_df.x = pd_reconstructed_df.x * x_coefficient
pd_reconstructed_df.y = pd_reconstructed_df.y * y_coefficient
pd_reconstructed_df.z = pd_reconstructed_df.z * z_coefficient

In [7]:
pd_reconstructed_df

,x,y,z
0,209.096006,698.785201,720.0
1,209.096006,698.991942,720.0
2,209.096006,699.198683,720.0
3,209.290876,698.785201,720.0
4,209.290876,698.991942,720.0
...,...,...,...
20325942,836.968634,520.160818,0.0
20325943,836.968634,520.367559,0.0
20325944,837.163504,519.954077,0.0
20325945,837.163504,520.160818,0.0


# PyVista

In [11]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pd_reconstructed_df.values)

In [14]:
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# Open3d

In [13]:
pcd = pv.PolyData (
	pd_reconstructed_df.loc [
		pd_reconstructed_df.z.between (4_000, 10_600_000)
	].values
	)

In [14]:
pcd_o3d = o3d.geometry.PointCloud ()
pcd_o3d.points = o3d.utility.Vector3dVector (
	pd_reconstructed_df.loc[
		pd_reconstructed_df.z < 400
	].values
)

In [15]:
pcd_o3d

PointCloud with 2681800 points.

In [41]:
o3d.io.write_point_cloud(
		filename = 'PCD_rec_test.ply',
		pointcloud = pcd_o3d
		)

True

In [16]:
o3d.visualization.draw_geometries ([pcd_o3d])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [9]:
pcd_download = o3d.geometry.PointCloud ()
pcd_download.points = o3d.utility.Vector3dVector (pd_reconstructed_df.values)

o3d.io.write_point_cloud ('layers_pcd.ply', pcd_download)

True